In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import zipfile
# from tqdm import tqdm
# import os

# # Path to the zip file and extraction directory
# zip_path = '/content/drive/MyDrive/AIMusicGenerator/lakh_midi_dataset.zip'
# extract_path = '/content/drive/MyDrive/AIMusicGenerator/lakh_music_dataset/'

# try:
#     # Using tqdm for progress bar
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         # Get the list of files
#         file_list = zip_ref.namelist()
#         total_files = len(file_list)

#         # Extract files with a progress bar
#         with tqdm(total=total_files, desc="Extracting Files", unit="file") as pbar:
#             for file in file_list:
#                 zip_ref.extract(file, extract_path)
#                 pbar.update(1)

#     print("File unzipped successfully!")
# except FileNotFoundError:
#     print(f"File not found: {zip_path}. Please check the path and try again.")
# except Exception as e:
#     print(f"An error occurred: {e}")


Extracting Files: 100%|██████████| 38868/38868 [07:47<00:00, 83.07file/s]

File unzipped successfully!


In [ ]:
!pip install music21

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.1 MB/s eta 0:00:00


In [ ]:
import music21
import os
import random
import numpy as np
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical

# Function to parse MIDI files and extract notes/chords
def get_notes(data_path, max_files=None):
    """
    Parse MIDI files to extract notes and chords.

    Parameters:
        data_path (str): Path to the dataset directory.
        max_files (int): Maximum number of MIDI files to process.

    Returns:
        notes (list): List of extracted notes or chords.
    """
    notes = []
    midi_files = []

    # Traverse the directory recursively to find all .mid files
    for root, _, files in os.walk(data_path):
        for file in files:
            if file.endswith(".mid"):
                midi_files.append(os.path.join(root, file))

    if len(midi_files) == 0:
        raise FileNotFoundError(f"No MIDI files found in {data_path}. Check the dataset structure.")

    # Select a subset of files if max_files is specified
    if max_files:
        midi_files = random.sample(midi_files, min(max_files, len(midi_files)))

    # Use tqdm for progress tracking
    for file in tqdm(midi_files, desc="Processing MIDI files"):
        try:
            midi = music21.converter.parse(file)
        except Exception as e:
            print(f"Error reading {file}: {e}")
            continue

        try:
            parts = music21.instrument.partitionByInstrument(midi)
            notes_to_parse = parts.parts[0].recurse() if parts else midi.flat.notes
        except Exception as e:
            print(f"Error parsing {file}: {e}")
            continue

        for element in notes_to_parse:
            if isinstance(element, music21.note.Note):  # Single note
                notes.append(str(element.pitch))
            elif isinstance(element, music21.chord.Chord):  # Chord
                # Identify as a chord and sort pitches
                chord_notes = '.'.join(str(n) for n in sorted(element.normalOrder))
                notes.append(chord_notes)

    return notes

# Preprocess MIDI files
data_path = '/content/drive/MyDrive/AIMusicGenerator/lakh_music_dataset/'
max_files_to_process = 50  # Limit the number of files to process
print(f"Extracting notes from up to {max_files_to_process} MIDI files...")
notes = get_notes(data_path, max_files=max_files_to_process)

if len(notes) == 0:
    raise ValueError("No notes were extracted from the dataset. Check the dataset and parsing logic.")

# Save unique notes and encode
unique_notes = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(unique_notes)}
sequence_length = 100

# Prepare sequences
input_sequences = []
output_notes = []

print("Preparing input sequences...")
for i in tqdm(range(len(notes) - sequence_length), desc="Creating sequences"):
    input_sequences.append([note_to_int[n] for n in notes[i:i + sequence_length]])
    output_notes.append(note_to_int[notes[i + sequence_length]])

n_vocab = len(unique_notes)

if len(input_sequences) == 0:
    raise ValueError("No input sequences could be created. Check your data and sequence length.")

# Reshape and normalize
print("Normalizing and reshaping data...")
X = np.reshape(input_sequences, (len(input_sequences), sequence_length, 1)) / n_vocab
y = to_categorical(output_notes, num_classes=n_vocab)

print("Data processing complete.")
print(f"Number of sequences: {len(input_sequences)}")
print(f"Vocabulary size: {n_vocab}")


Extracting notes from up to 50 MIDI files...


Processing MIDI files:   4%|▍         | 2/50 [00:06<02:10,  2.72s/it]

Error reading /content/drive/MyDrive/AIMusicGenerator/lakh_music_dataset/__MACOSX/lakh_midi_dataset/Ranieri/._Rose_rosse.1.mid: badly formatted midi bytes, got: b'\x00\x05\x16\x07\x00\x02\x00\x00Mac OS X    '


Processing MIDI files:  10%|█         | 5/50 [00:11<01:06,  1.48s/it]

Error reading /content/drive/MyDrive/AIMusicGenerator/lakh_music_dataset/__MACOSX/lakh_midi_dataset/Air_Supply/._She_Never_Heard_Me_Call.mid: badly formatted midi bytes, got: b'\x00\x05\x16\x07\x00\x02\x00\x00Mac OS X    '
Error reading /content/drive/MyDrive/AIMusicGenerator/lakh_music_dataset/__MACOSX/lakh_midi_dataset/Neil_Young/._Southern_Man.mid: badly formatted midi bytes, got: b'\x00\x05\x16\x07\x00\x02\x00\x00Mac OS X    '


...

Error reading /content/drive/MyDrive/AIMusicGenerator/lakh_music_dataset/__MACOSX/lakh_midi_dataset/DJ_Quicksilver/._Bellisima.mid: badly formatted midi bytes, got: b'\x00\x05\x16\x07\x00\x02\x00\x00Mac OS X    '


Processing MIDI files: 100%|██████████| 50/50 [01:55<00:00,  2.31s/it]


Error reading /content/drive/MyDrive/AIMusicGenerator/lakh_music_dataset/__MACOSX/lakh_midi_dataset/Hasselhoff_David/._Do_the_Limbo_Dance.2.mid: badly formatted midi bytes, got: b'\x00\x05\x16\x07\x00\x02\x00\x00Mac OS X    '
Error reading /content/drive/MyDrive/AIMusicGenerator/lakh_music_dataset/__MACOSX/lakh_midi_dataset/Naked_Eyes/._Theres_Always_Something_There_to_Remind_Me.mid: badly formatted midi bytes, got: b'\x00\x05\x16\x07\x00\x02\x00\x00Mac OS X    '
Preparing input sequences...


Creating sequences: 100%|██████████| 2256/2256 [00:00<00:00, 125588.63it/s]

Normalizing and reshaping data...
Data processing complete.
Number of sequences: 2256
Vocabulary size: 53


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation

# Define the LSTM model
def create_model(input_shape, n_vocab):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_vocab, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Create the model
model = create_model((X.shape[1], X.shape[2]), n_vocab)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 lstm_1 (LSTM)               (None, 512)               2099200   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 53)                13621     
                                                                 
Total params: 3296821 (12.58 MB)
Trainable params: 32968

In [ ]:
# Train the model
model.fit(X, y, epochs=100, batch_size=64)
save_path = '/content/drive/MyDrive/AIMusicGenerator/music_generic/music_generator.h5'
model.save(save_path)


Epoch 1/100
36/36 [==============================] - 35s 889ms/step - loss: 3.2259
Epoch 2/100
36/36 [==============================] - 32s 894ms/step - loss: 3.0654
Epoch 3/100
36/36 [==============================] - 32s 878ms/step - loss: 2.9571
Epoch 4/100
36/36 [==============================] - 32s 879ms/step - loss: 2.8584
Epoch 5/100
36/36 [==============================] - 32s 889ms/step - loss: 2.8594
Epoch 6/100
36/36 [==============================] - 32s 884ms/step - loss: 2.8839
Epoch 7/100
36/36 [==============================] - 32s 881ms/step - loss: 2.9791
Epoch 8/100
36/36 [==============================] - 32s 876ms/step - loss: 2.8338
Epoch 9/100
36/36 [==============================] - 32s 881ms/step - loss: 2.6102
Epoch 10/100
36/36 [==============================] - 32s 883ms/step - loss: 2.6464
Epoch 11/100
36/36 [==============================] - 32s 876ms/step - loss: 2.3662
Epoch 12/100
36/36 [==============================] - 32s 885ms/step - loss: 2.1812
E

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Generate music
import random

def generate_music(model, int_to_note, sequence_length, n_vocab, num_notes=500):
    # Pick a random seed sequence
    start = np.random.randint(0, len(input_sequences) - 1)
    pattern = input_sequences[start]
    prediction_output = []

    # Generate notes
    for _ in range(num_notes):
        input_seq = np.reshape(pattern, (1, sequence_length, 1)) / n_vocab
        prediction = model.predict(input_seq, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern.append(index)
        pattern = pattern[1:]
    return prediction_output

# Load saved model
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/AIMusicGenerator/music_generic/music_generator.h5')
int_to_note = {number: note for note, number in note_to_int.items()}
generated_notes = generate_music(model, int_to_note, sequence_length, n_vocab)


In [ ]:
def create_midi(prediction_output, output_file="output.mid"):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if '.' in pattern or pattern.isdigit():
            # It's a chord
            notes_in_chord = pattern.split('.')
            chord_notes = [music21.note.Note(int(n)) for n in notes_in_chord]
            for note in chord_notes:
                note.storedInstrument = music21.instrument.Piano()
            new_chord = music21.chord.Chord(chord_notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            # It's a note
            new_note = music21.note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = music21.instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5

    midi_stream = music21.stream.Stream(output_notes)
    midi_stream.write("midi", fp=output_file)

# Save generated music as MIDI
out_file = '/content/drive/MyDrive/AIMusicGenerator/music_generic/output.mid'
create_midi(generated_notes, output_file = out_file)
